# Using a genetic algorithm to train an agent in the Game Of Life

Types of agent:
    - Glider
    - 
    

    
Method:
    - Define a fitness function for agent
    - Generate a certain number of dna
    - Test the fitness of all the dna
    - Pick the best dna from the pool, mutate them
    - Repeat until you have a satisfactory agent that can perform the task specified

In [1]:
import cv2 as cv
import numpy as np
from dna import *
from gol import *
from fitness import *
from main import *
from dcp_fitness import *

run_dcp = True

if run_dcp:
    # ! pip install -q git+https://github.com/Distributive-Network/Bifrost
    from bifrost import dcp  
    print("bifrost import good!!!")

Warning! Hook 'shutdown_hook' is not one of ['editor', 'synchronize_with_editor', 'show_in_pager', 'pre_prompt_hook', 'clipboard_get']
proceeding with currently installed version of dcp-client
bifrost import good!!!


## Training an agent to explode

In [2]:
board_dim = (100, 100)
agent_dim = (10, 10)
starting_pos = (45, 45)
mutation_rate = 0.05
max_frames = 5000

best_of_each_gen = []
num_generation = 2


# Create the population
num_agents = 100
# population: [(agent:DNA, score:int)]
population = {}
# Create the initial population
for i in range(num_agents):
    agent = DNA(agent_dim)
    score = 0
    population[agent] = score

for i in range(num_generation):
    print("Generation: ", i)
    best_agent = (None, 0)    

    # Score the population
    ## Hook this up to the distributive end
    
    if run_dcp:
        
        agents = list(population.keys())
        input_set = []
        for agent in agents:
            # agent, max_frames, board_dim, starting_pos = input
            input_ = agent.dna, max_frames, board_dim, starting_pos
            input_set.append(input_)
            
        ## Comment our for dcp
#         scores = []
#         for i in input_set:
#             score = dcp_fitness(i)
#             scores.append(score)
        ## Comment out for dcp
            
        ## uncomment for dcp
        job = dcp.compute_for(input_set, dcp_fitness)
        job.requires('numpy')
        job.compute_groups = [{'joinKey': 'insight', 'joinSecret': 'dcp'}]
        job.public['name'] = "Training Game of Life agent!"
        
        scores = job.exec()
        ## uncomment for dcp
        
        for i in range(len(scores)):
            score = scores[i]
            agent = agents[i]
            population[agent] = score
            if scores[i] > best_agent[1]:
                best_agent = (agent, score)
        
        
    else:
        
        for agent in population.keys():
            score = explosion_fitness(agent, board_dim=board_dim, starting_pos=starting_pos)
            population[agent] = score
            if score > best_agent[1]:
                best_agent = (agent, score)
    

    best_of_each_gen.append(best_agent[0])
    print("Best score: ", best_agent[1])

    # print(population)

    population = next_generation(population, mutation_rate=mutation_rate)



Generation:  0
State : exec
State : init
State : preauth
State : deploying
State : listeners
State : compute-groups
State : uploading
State : deployed
Accepted : 0x69B36938F840175E2c3413161c5e165dE254a0af
Computed : 1.00%
Computed : 2.00%
Computed : 3.00%
Computed : 4.00%
Computed : 5.00%
Computed : 6.00%
Computed : 7.00%
Computed : 8.00%
Computed : 9.00%
Computed : 10.00%
Computed : 11.00%
Computed : 12.00%
Computed : 13.00%
Computed : 14.00%
Computed : 15.00%
Computed : 16.00%
Computed : 17.00%
Computed : 18.00%
Computed : 19.00%
Computed : 20.00%
Computed : 21.00%
Computed : 22.00%
Computed : 23.00%
Computed : 24.00%
Computed : 25.00%
Computed : 26.00%
Computed : 27.00%
Computed : 28.00%
Computed : 29.00%
Computed : 30.00%
Computed : 31.00%
Computed : 32.00%
Computed : 33.00%
Computed : 34.00%
Computed : 35.00%
Computed : 36.00%
Computed : 37.00%
Computed : 38.00%
Computed : 39.00%
Computed : 40.00%
Computed : 41.00%
Computed : 42.00%
Computed : 43.00%
Computed : 44.00%
Computed : 4

In [5]:
gen = 1
    
for agent in best_of_each_gen:

    # Create the game of life baord 
    gol_board = GOL(board_dim)

    # Create a random agent
    # Define the position we would like to place the agent
    gol_board.add_agent(starting_pos, agent)

    animate_board(gol_board, frame_rate = 10, text="gen: " + str(gen))
    gen += 1

In [ ]:
# pip install -q git+https://github.com/Distributive-Network/Bifrost


# DCP MODULE
from bifrost import dcp

##################
# INPUT SET
##################
b_set = range(12)
##################
# WORK FUNCTION
##################
"Compute left Riemann sum of exp(-x^2) from 0 to b with N subintervals."
def riemann_sum(b,N=100000):

  # packages required inside the work function must be imported inside the work function
  import numpy as np

  x = np.linspace(0,b,N+1)
  x_left_endpoints = x[:-1]
  Delta_x = b/N
  I = Delta_x * np.sum(np.exp(-x_left_endpoints**2))

  return I

##################
# JOB & CONFIG
##################
job = dcp.compute_for(b_set, riemann_sum)
job.requires('numpy')
job.compute_groups = [{'joinKey': 'demo', 'joinSecret': 'dcp'}]
job.public['name'] = "Numpy Riemann Sums Job, on DCP!"

##################
# EXEC
##################
results = job.exec()


print(results)



In [8]:
cv.destroyAllWindows()
